# <b>Trash Recognition Using Deep Learning Model</b>

## <b>Problem Statement</b>

1. How can an algorithmic model identify waste objects based on their categories?
2. Can models based on deep learning recognize trash objects?

In [9]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
dataset_path = 'trashnet\dataset-resized'

img_paths = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            img_paths.append(os.path.join(root, file))

labels = []
for path in img_paths:
    label = os.path.basename(os.path.dirname(path))
    labels.append(label)

## <b>Exploratory Image Analysis</b>

In [ ]:
# Mengonversi label menjadi bentuk one-hot encoding
label_set = list(labels)
label_dict = {label: i for i, label in enumerate(label_set)}
labels_one_hot = [tf.one_hot(label_dict[label], len(label_set)) for label in labels]

In [ ]:
import tensorflow as tf

def load_and_augmented_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [200, 200])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta = 0.1)
    image = tf.image.random_contrast(image, lower = 0.9, upper = 1.1)
    image /= 255.0
    return image, label

batch_size = 128
dataset = tf.data.Dataset.from_tensor_slices((img_paths, labels_one_hot))
dataset = dataset.map(load_and_augmented_image)
dataset = dataset.shuffle(buffer_size = len(img_paths))
dataset = dataset.batch(batch_size)

# Visualisasi beberapa gambar dari dataset
plt.figure(figsize=(15, 15))
for images, labels in dataset.take(1):
  for i in range(12):
    ax = plt.subplot(5, 3, i + 1)
    plt.imshow(images[i])
    label = label_set[tf.argmax(labels[i])]
    plt.title(label)
    plt.axis("off")

In [ ]:
# Memisahkan dataset menjadi dataset pelatihan dan validasi
train_size = int(0.8 * len(img_paths))
val_size = len(img_paths) - train_size

train_dataset = dataset.take(train_size // batch_size)
val_dataset = dataset.skip(train_size // batch_size)

print(f"Jumlah Data Pelatihan (Train): {train_size}")
print(f"Jumlah data Validasi (Test): {val_size}")

In [ ]:
# Modelling
model = Sequential()

model.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(label_set), activation='softmax'))

model.summary()

In [ ]:
# Create EarlyStopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
# Visualisasi gambaran model CNN
import visualkeras

visualkeras.layered_view(model,
                         to_file='model_plot_cnn.png',
                         show_dimension=True,
                         legend=True) # write to disk

In [ ]:
# Kompilasi model dan pelatihan model
model.compile(Adam(learning_rate=0.01, weight_decay=0.04),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Histori model
EPOCHS = 25

history = model.fit(train_dataset,
                    steps_per_epoch = train_size // batch_size,
                    epochs = EPOCHS,
                    batch_size = batch_size,
                    validation_data = (val_dataset),
                    verbose = 2)